In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./kaggle/input/digit-recognizer'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./kaggle/input/digit-recognizer\sample_submission.csv
./kaggle/input/digit-recognizer\test.csv
./kaggle/input/digit-recognizer\train.csv


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# --- Configuration ---
BATCH_SIZE = 64
EPOCHS = 30  # Increase to 50+ for maximum accuracy
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

# Set reproducibility
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

print(f"Using device: {DEVICE}")

Using device: cpu


In [4]:
class DigitDataset(Dataset):
    def __init__(self, data, transform=None, is_test=False):
        self.transform = transform
        self.is_test = is_test
        
        if self.is_test:
            # Test data has no labels, just pixels
            self.X = data.values.reshape((-1, 28, 28, 1)).astype(np.uint8)
            self.y = None
        else:
            # Train data: Col 0 is label, rest are pixels
            self.y = torch.from_numpy(data.iloc[:, 0].values).type(torch.LongTensor)
            self.X = data.iloc[:, 1:].values.reshape((-1, 28, 28, 1)).astype(np.uint8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Convert numpy image to PIL-like format for transforms, then to Tensor
        image = self.X[idx]
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image
        else:
            return image, self.y[idx]

In [5]:
print("Loading data...")
train_df = pd.read_csv('./kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('./kaggle/input/digit-recognizer/test.csv')

# Split train set into Train/Validation to monitor performance
# We use a small validation set to keep more data for training
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=SEED)

# Data Augmentation (Crucial for >99% Accuracy)
# We add rotation, shifting, and zooming to make the model robust
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(degrees=10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]) # Normalize to [-1, 1]
])

val_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = DigitDataset(train_data, transform=train_transforms)
val_dataset = DigitDataset(val_data, transform=val_transforms)
test_dataset = DigitDataset(test_df, transform=val_transforms, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading data...


In [6]:
class DigitRecognizerNet(nn.Module):
    def __init__(self):
        super(DigitRecognizerNet, self).__init__()
        
        # Block 1
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )
        
        # Block 2
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )

        # Fully Connected Layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10) # 10 Output classes (Digits 0-9)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc(x)
        return x

model = DigitRecognizerNet().to(DEVICE)
optimizer = optim.RMSprop(model.parameters(), lr=LEARNING_RATE, alpha=0.9)
criterion = nn.CrossEntropyLoss()

# Learning Rate Scheduler: Reduce LR when validation accuracy plateaus

scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

In [7]:
print("Starting Training...")

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_acc = correct / total
    scheduler.step(val_acc) # Adjust LR based on validation accuracy
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss/len(train_loader):.4f} | Val Acc: {val_acc:.4f}")

Starting Training...
Epoch 1/30 | Loss: 0.2337 | Val Acc: 0.9852
Epoch 2/30 | Loss: 0.1042 | Val Acc: 0.9874
Epoch 3/30 | Loss: 0.0909 | Val Acc: 0.9905
Epoch 4/30 | Loss: 0.0812 | Val Acc: 0.9919
Epoch 5/30 | Loss: 0.0763 | Val Acc: 0.9914
Epoch 6/30 | Loss: 0.0737 | Val Acc: 0.9933
Epoch 7/30 | Loss: 0.0708 | Val Acc: 0.9938
Epoch 8/30 | Loss: 0.0713 | Val Acc: 0.9929
Epoch 9/30 | Loss: 0.0687 | Val Acc: 0.9945
Epoch 10/30 | Loss: 0.0632 | Val Acc: 0.9936
Epoch 11/30 | Loss: 0.0685 | Val Acc: 0.9938
Epoch 12/30 | Loss: 0.0620 | Val Acc: 0.9943
Epoch 13/30 | Loss: 0.0654 | Val Acc: 0.9943
Epoch 14/30 | Loss: 0.0546 | Val Acc: 0.9955
Epoch 15/30 | Loss: 0.0544 | Val Acc: 0.9952
Epoch 16/30 | Loss: 0.0517 | Val Acc: 0.9955
Epoch 17/30 | Loss: 0.0521 | Val Acc: 0.9940
Epoch 18/30 | Loss: 0.0514 | Val Acc: 0.9952
Epoch 19/30 | Loss: 0.0497 | Val Acc: 0.9955
Epoch 20/30 | Loss: 0.0421 | Val Acc: 0.9955
Epoch 21/30 | Loss: 0.0449 | Val Acc: 0.9955
Epoch 22/30 | Loss: 0.0453 | Val Acc: 0.995

In [8]:
print("Generating predictions for test set...")
model.eval()
predictions = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

submission = pd.DataFrame({
    "ImageId": range(1, len(predictions) + 1),
    "Label": predictions
})

submission.to_csv("./kaggle/input/digit-recognizer/submission.csv", index=False)
print("submission.csv saved successfully!")

Generating predictions for test set...
submission.csv saved successfully!
